In [ ]:
# A default setup cell.
# It imports environment variables, define 'devtools.debug" as a buildins, set PYTHONPATH, and code auto-reload
# Copy it in other Notebooks


from dotenv import load_dotenv
from rich import print

load_dotenv(verbose=True)

%load_ext autoreload
%autoreload 2
%reset -f

In [ ]:
import pandas as pd
from genai_tk.core.llm_factory import LlmFactory
from genai_tk.utils.config_mngr import global_config
from smolagents import (
    LiteLLMModel,
)

In [ ]:
MODEL_ID = None
model_name = LlmFactory(llm_id=MODEL_ID).get_litellm_model_name()
llm = LiteLLMModel(model_id=model_name)
print(model_name)

In [ ]:
DATASETS = {
    "country CO2 emissions": (
        "Country daily CO2 emissions from 2019 to 2024",
        "carbon_global_dataset/carbon_global.csv",
    ),
    "cities CO2 emissions": (
        "Major cities daily CO2 emissions from 2019 to 2024",
        "carbon_global_dataset/carbon_cities.csv",
    ),
    "GDP": ("Countries GDP per year", "gdp/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_2.csv"),
    "country population": ("Country populatipn from 1960 to 2023", "world_population/world_population.csv"),
}


def get_data(dataset: str) -> pd.DataFrame:
    """Load and cache CO2 emissions data from configured dataset.

    Returns:
        DataFrame containing emissions data by country and sector.
    """
    description, data_set = DATASETS[dataset]
    data_file = global_config().get_dir_path("datasets_root") / data_set
    print(data_file, description)
    assert data_file.exists(), f"file  not found: {data_file}"
    if data_file.name.endswith(".csv"):
        df = pd.read_csv(data_file, decimal=",")
    else:
        df = pd.read_excel(data_file)
    return df

In [ ]:
import folium

m = folium.Map([45.35, -121.6972], zoom_start=12)

folium.Marker(
    location=[45.3288, -121.6625],
    tooltip="Click me!",
    popup="Mt. Hood Meadows",
    icon=folium.Icon(icon="cloud"),
).add_to(m)

folium.Marker(
    location=[45.3311, -121.7113],
    tooltip="Click me!",
    popup="Timberline Lodge",
    icon=folium.Icon(color="green"),
).add_to(m)
m

In [ ]:
m.save("map.html")

In [ ]:
import pandas
import requests

state_geo = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
).json()
state_data = pandas.read_csv(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_unemployment_oct_2012.csv"
)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
m.save("map1.html")